In [ ]:
import os
import pathlib
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset,DataLoader
from torch import nn
from sklearn.model_selection import train_test_split

import json
import multiprocessing
import glob

from keras.utils import np_utils


from tensorflow import keras
from keras.layers import *
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
import datetime
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras import optimizers
from keras.layers import Reshape

In [ ]:
class EEGLoader(Dataset):
  def __init__(self,root):
    super(EEGLoader,self).__init__()
    self.root = root
    self.name2label = {}
    for name in sorted(os.listdir(os.path.join(root))):
      if not os.path.isdir(os.path.join(root,name)):
        continue
      self.name2label[name] = len(self.name2label.keys())
    print(self.name2label)

    # no enough data, so split two dataset manually. Just in case use shuffle,split will get only one or two samples for test 
    # assert len(data) == len(labels)


  # def load_csv(self,filename):
  #     data  = []
  #     for name in self.name2label.keys():
  #       print("name",name)
  #       data +=glob.glob(os.path.join(self.root,name,'*.csv'))
  #       # 。split(os.sep)[-2]
  #     print(len(data),data)


  def load_csv(self,foldername):
      # data  = []
      data =glob.glob(os.path.join(self.root,foldername,'*.csv'))
      # 。split(os.sep)[-2]
      print(len(data),data)

      res = []
      unusedChan = []
      for file in data:
        EEGMatrix = pd.read_csv(file,header=None).values
        EEGwithoutConstantValue = []
        for index,channel in enumerate(EEGMatrix):
          # print("channel len",len(channel))
          # for testing since number of each acquisition's unique channels without 134733100 is different. get min number 29 rows first. Do data preprocessing to reduce channels to 17 later.
          if 134733100.0 in channel:
            if index not in unusedChan:
              unusedChan.append(index)
            continue
          EEGwithoutConstantValue.append(channel)
        # print(np.array(EEGwithoutConstantValue).shape)
        EEGwithoutConstantValue = EEGwithoutConstantValue[:24]
        # print(np.array(EEGwithoutConstantValue).shape)
        res.append(EEGwithoutConstantValue)
      print("unuse channel",unusedChan)
      return res
  

  def __len__(self):
      pass

  def __getitem__(self,idx):
      pass
data = EEGLoader('/content/drive/MyDrive/EEG')

resting_list = data.load_csv("01-health")
music_list = data.load_csv("08-health")

print(np.array(resting_list).shape)
print(np.array(music_list).shape)


{'01': 0, '01-health': 1, '01-previous': 2, '08': 3, '08-health': 4, '08-previous': 5, '16': 6, '16-health': 7}
90 ['/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-3.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-4.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-1.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-2.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-7.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-6.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-8.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-5.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-9.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 19-rawTrigger1-10.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 20-rawTrigger1-3.csv', '/content/drive/MyDrive/EEG/01-health/Acquisition 20-rawTrigger1-4.csv', '/conte

In [ ]:

# train
x_train_a = resting_list[:60]
# .reshape((300,160,24))
x_train_a  = np.array(x_train_a)
x_train_a.shape

x_train_b = music_list[:150]
# .reshape((300,160,24))
x_train_b = np.array(x_train_b)
x_train = np.vstack((x_train_a,x_train_b))
x_train.shape

x_train /= 200
x_train = x_train.astype("float32")

# test
x_test_a = resting_list[60:]
# .reshape((150,160,24))
x_test_a  = np.array(x_test_a)
x_test_a.shape
print(len(x_test_a))

x_test_b = music_list[150:]
# .reshape((150,160,24))
x_test_b = np.array(x_test_b)
x_test = np.vstack((x_test_a,x_test_b))
x_test.shape

x_test /= 200
x_test = x_test.astype("float32")
print(len(x_test))
# labels
y_train = np.append(np.zeros(len(x_train_a),dtype=int), np.ones(len(x_train_b),dtype=int))
y_train.shape

y_train = np_utils.to_categorical(y_train,2)
y_train = y_train.astype("float32")
y_train

y_test = np.append(np.zeros(len(x_test_a),dtype=int), np.ones(len(x_test_b),dtype=int))
y_test.shape


y_test = np_utils.to_categorical(y_test,2)
y_test = y_test.astype("float32")

30
72


In [ ]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)
print(x_train.shape)
print(x_test.shape)

(210, 8000, 24)
(72, 8000, 24)


In [ ]:
from tensorflow.python.ops.gen_array_ops import Transpose
def residual_block(layer_input, filters):
  """Residual block described in paper"""
  d = Conv2D(filters, 3, activation='relu', padding='same')(layer_input)
  d = BatchNormalization(momentum=0.8)(d)
  d = Conv2D(filters, 3, activation='relu', padding='same')(d)
  d = BatchNormalization(momentum=0.8)(d)
  d = Add()([d, layer_input])
  return d




# cnn
# (192, 24, 8000) -> transpose 192 8000 24 -> reshape 192,20,400,24 
# 192,8000,2
# 变成(192,20,400,24)
inputs = Input(shape=(8000, 24))
c = Reshape((200,40,24))(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(c)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
c2 = residual_block(conv2, 64)
#pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
c2 = residual_block(c2, 64)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(c2)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
D1 = Flatten()(conv3)
D1 = Dense(128,activation='relu')(D1) #relu
D1 = Dense(32)(D1) 
Out = Dense(2, activation='softmax')(D1)

model = Model(inputs, Out)
model.summary()
# 0.0001
opt = optimizers.Adam(learning_rate=0.0001, decay=1e-6) #0.002， 0.0005
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 8000, 24)]   0           []                               
                                                                                                  
 reshape_13 (Reshape)           (None, 200, 40, 24)  0           ['input_14[0][0]']               
                                                                                                  
 conv2d_104 (Conv2D)            (None, 200, 40, 64)  13888       ['reshape_13[0][0]']             
                                                                                                  
 conv2d_105 (Conv2D)            (None, 200, 40, 64)  36928       ['conv2d_104[0][0]']             
                                                                                           

In [ ]:
history = model.fit(x_train,y_train,epochs=15,batch_size=3)

Epoch 1/15
70/70 [==============================] - 2s 21ms/step - loss: 88.0177 - accuracy: 0.7048
Epoch 2/15
70/70 [==============================] - 1s 20ms/step - loss: 14.3332 - accuracy: 0.7762
Epoch 3/15
70/70 [==============================] - 1s 20ms/step - loss: 7.2855 - accuracy: 0.7762
Epoch 4/15
70/70 [==============================] - 1s 20ms/step - loss: 3.1969 - accuracy: 0.8476
Epoch 5/15
70/70 [==============================] - 1s 20ms/step - loss: 2.5665 - accuracy: 0.8190
Epoch 6/15
70/70 [==============================] - 1s 20ms/step - loss: 0.9524 - accuracy: 0.8762
Epoch 7/15
70/70 [==============================] - 1s 20ms/step - loss: 0.4049 - accuracy: 0.8857
Epoch 8/15
70/70 [==============================] - 1s 20ms/step - loss: 0.1610 - accuracy: 0.9333
Epoch 9/15
70/70 [==============================] - 1s 20ms/step - loss: 0.4330 - accuracy: 0.8905
Epoch 10/15
70/70 [==============================] - 1s 20ms/step - loss: 0.4088 - accuracy: 0.8952
Epoch 1

In [ ]:
y_pred = model.predict(x_test)

3/3 [==============================] - 0s 41ms/step


In [ ]:
y_pred

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
pred

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0,

In [ ]:
pred = np.int64(y_pred>0.5)
y_label = []
for v in y_test:
  y_label.append(v[0])
y_label=np.array(y_label,dtype=int)

pred_label = []
for v in pred:
  pred_label.append(v[0])
pred_label=np.array(pred_label,dtype=int)
print(y_label)
print(pred_label)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
f1_score(y_label,pred_label)

0.0

# Patient

In [ ]:
from torch.autograd.grad_mode import F
class EEGLoader(Dataset):
  def __init__(self,root):
    super(EEGLoader,self).__init__()
    self.root = root
    self.name2label = {}
    for name in sorted(os.listdir(os.path.join(root))):
      if not os.path.isdir(os.path.join(root,name)):
        continue
      self.name2label[name] = len(self.name2label.keys())
    print(self.name2label)

    # no enough data, so split two dataset manually. Just in case use shuffle,split will get only one or two samples for test 
    # assert len(data) == len(labels)


  # def load_csv(self,filename):
  #     data  = []
  #     for name in self.name2label.keys():
  #       print("name",name)
  #       data +=glob.glob(os.path.join(self.root,name,'*.csv'))
  #       # 。split(os.sep)[-2]
  #     print(len(data),data)


  def load_csv(self,foldername):
      # data  = []
      data =glob.glob(os.path.join(self.root,foldername,'*.csv'))
      # 。split(os.sep)[-2]
      print(len(data),data)

      res = []
      unusedChan = []
      for file in data:
        # remove dead people
        # if "Acquisition 09" in file : # or "Acquisition 30" in file
        #   continue

        EEGMatrix = pd.read_csv(file,header=None).values
        EEGwithoutConstantValue = []
        for index,channel in enumerate(EEGMatrix):
          # print("channel len",len(channel))
          # for testing since number of each acquisition's unique channels without 134733100 is different. get min number 29 rows first. Do data preprocessing to reduce channels to 17 later.
          if 134733100.0 in channel:
            if index not in unusedChan:
              unusedChan.append(index)
            continue
          EEGwithoutConstantValue.append(channel)
        # print(np.array(EEGwithoutConstantValue).shape)
        EEGwithoutConstantValue = EEGwithoutConstantValue[:24]
        # print(np.array(EEGwithoutConstantValue).shape)
        res.append(EEGwithoutConstantValue)
      print("unuse channel",unusedChan)
      return res
  

  def __len__(self):
      pass

  def __getitem__(self,idx):
      pass
data = EEGLoader('/content/drive/MyDrive/EEG')

resting_list2 = data.load_csv("01")
music_list2 = data.load_csv("08")

print(np.array(resting_list2).shape)
print(np.array(music_list2).shape)

{'01': 0, '01-health': 1, '01-previous': 2, '08': 3, '08-health': 4, '08-previous': 5, '16': 6, '16-health': 7}
170 ['/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-1.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-2.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-4.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-5.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-3.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-8.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-7.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-9.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-6.csv', '/content/drive/MyDrive/EEG/01/Acquisition 09-rawTrigger1-10.csv', '/content/drive/MyDrive/EEG/01/Acquisition 10-rawTrigger1-1.csv', '/content/drive/MyDrive/EEG/01/Acquisition 10-rawTrigger1-2.csv', '/content/drive/MyDrive/EEG/01/Acquisition 10-rawTrigger1-4.csv', '/content/drive/MyDrive/

In [ ]:
# patients
# train
# (170, 24, 8000)
# (294, 24, 8000)
x_train_a = resting_list2[:100]
# .reshape((300,160,24))
x_train_a  = np.array(x_train_a)
x_train_a.shape

x_train_b = music_list2[:200]
# .reshape((300,160,24))
x_train_b = np.array(x_train_b)
x_train = np.vstack((x_train_a,x_train_b))
x_train.shape

x_train /= 200
x_train = x_train.astype("float32")

# test
x_test_a = resting_list2[100:]
# .reshape((150,160,24))
x_test_a  = np.array(x_test_a)
x_test_a.shape
print(len(x_test_a))

x_test_b = music_list2[200:]
# .reshape((150,160,24))
x_test_b = np.array(x_test_b)
x_test = np.vstack((x_test_a,x_test_b))
x_test.shape

x_test /= 200
x_test = x_test.astype("float32")
print(len(x_test))
# labels
y_train = np.append(np.zeros(len(x_train_a),dtype=int), np.ones(len(x_train_b),dtype=int))
y_train.shape

y_train = np_utils.to_categorical(y_train,2)
y_train = y_train.astype("float32")
y_train

y_test = np.append(np.zeros(len(x_test_a),dtype=int), np.ones(len(x_test_b),dtype=int))
y_test.shape


y_test = np_utils.to_categorical(y_test,2)
y_test = y_test.astype("float32")

70
164


In [ ]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)
print(x_train.shape)
print(x_test.shape)

(300, 8000, 24)
(164, 8000, 24)


In [ ]:
from tensorflow.python.ops.gen_array_ops import Transpose
def residual_block(layer_input, filters):
  """Residual block described in paper"""
  d = Conv2D(filters, 3, activation='relu', padding='same')(layer_input)
  d = BatchNormalization(momentum=0.8)(d)
  d = Conv2D(filters, 3, activation='relu', padding='same')(d)
  d = BatchNormalization(momentum=0.8)(d)
  d = Add()([d, layer_input])
  return d




# cnn
# (192, 24, 8000) -> transpose 192 8000 24 -> reshape 192,20,400,24 
# 192,8000,2
# 变成(192,20,400,24)
inputs = Input(shape=(8000, 24))
c = Reshape((200,40,24))(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(c)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
c2 = residual_block(conv2, 64)
#pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
c2 = residual_block(c2, 64)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(c2)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
D1 = Flatten()(conv3)
D1 = Dense(128,activation='relu')(D1) #relu
D1 = Dense(32)(D1) 
Out = Dense(2, activation='softmax')(D1)

model = Model(inputs, Out)
model.summary()

opt = optimizers.Adam(learning_rate=0.0001, decay=1e-6) #0.002， 0.0005
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 8000, 24)]   0           []                               
                                                                                                  
 reshape_19 (Reshape)           (None, 200, 40, 24)  0           ['input_20[0][0]']               
                                                                                                  
 conv2d_152 (Conv2D)            (None, 200, 40, 64)  13888       ['reshape_19[0][0]']             
                                                                                                  
 conv2d_153 (Conv2D)            (None, 200, 40, 64)  36928       ['conv2d_152[0][0]']             
                                                                                           

In [ ]:
history = model.fit(x_train,y_train,epochs=15,batch_size=3)

Epoch 1/15
100/100 [==============================] - 3s 20ms/step - loss: 41.9009 - accuracy: 0.6500
Epoch 2/15
100/100 [==============================] - 2s 20ms/step - loss: 5.7568 - accuracy: 0.8033
Epoch 3/15
100/100 [==============================] - 2s 20ms/step - loss: 2.0650 - accuracy: 0.8667
Epoch 4/15
100/100 [==============================] - 2s 20ms/step - loss: 2.1820 - accuracy: 0.8700
Epoch 5/15
100/100 [==============================] - 2s 20ms/step - loss: 6.5340 - accuracy: 0.7800
Epoch 6/15
100/100 [==============================] - 2s 20ms/step - loss: 1.2491 - accuracy: 0.8733
Epoch 7/15
100/100 [==============================] - 2s 20ms/step - loss: 0.3010 - accuracy: 0.9267
Epoch 8/15
100/100 [==============================] - 2s 20ms/step - loss: 0.2383 - accuracy: 0.9200
Epoch 9/15
100/100 [==============================] - 2s 20ms/step - loss: 0.3455 - accuracy: 0.9333
Epoch 10/15
100/100 [==============================] - 2s 20ms/step - loss: 0.3914 - accur

In [ ]:
model.save("failModel.h5")

In [ ]:
from sklearn.metrics import f1_score
y_pred = model.predict(x_test)

6/6 [==============================] - 1s 81ms/step


In [ ]:
ls=["Acquisition 09","Acquisition 19","Acquisition 29"]
for file in ls:
  if "Acquisition 09" in file or "Acquisition 19" in file:
    continue
  print(file) 

Acquisition 29


In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
roc_curve()

In [ ]:
pred = np.int64(y_pred>0.5)
y_label = []
for v in y_test:
  y_label.append(v[0])
y_label=np.array(y_label,dtype=int)

pred_label = []
for v in pred:
  pred_label.append(v[0])
pred_label=np.array(pred_label,dtype=int)
pred_label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
f1_score(y_label,pred_label)

0.0

In [ ]:
results = model.evaluate(x_test, y_test)

5/5 [==============================] - 0s 36ms/step - loss: 4.8479 - accuracy: 0.4692


In [ ]:
y_pred

array([[0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [8.57976496e-01, 1.42023534e-01],
       [8.84385884e-01, 1.15614109e-01],
       [8.74130368e-01, 1.25869662e-01],
       [8.67488503e-01, 1.32511482e-01],
       [8.47589731e-01, 1.52410328e-01],
       [8.25873137e-01, 1.74126893e-01],
       [8.38481128e-01, 1.61518916e-01],
       [8.11457574e-01, 1.88542455e-01],
       [7.82650650e-01, 2.17349380e-01],
       [7.98605025e-01, 2.01394916e-01],
       [2.06805652e-07, 9.99999762e-01],
       [1.17740299e-07, 9.99999881e-01],
       [2.32861566e-07, 9.99999762e-01],
       [2.05017500e-07, 9.99999762e-01],
       [1.615602

In [ ]:
2.05017500e-07

2.050175e-07

In [ ]:
from tensorflow.python.ops.gen_array_ops import Transpose
def residual_block(layer_input, filters):
  """Residual block described in paper"""
  d = Conv2D(filters, 3, activation='relu', padding='same')(layer_input)
  d = BatchNormalization(momentum=0.8)(d)
  d = Conv2D(filters, 3, activation='relu', padding='same')(d)
  d = BatchNormalization(momentum=0.8)(d)
  d = Add()([d, layer_input])
  return d




# cnn
# (192, 24, 8000) -> transpose 192 8000 24 -> reshape 192,20,400,24 
# 192,8000,2
# 变成(192,20,400,24)
inputs = Input(shape=(8000, 24))
c = Reshape((200,40,24))(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(c)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
c2 = residual_block(conv2, 64)
#pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
c2 = residual_block(c2, 64)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(c2)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
D1 = Flatten()(conv3)
D1 = Dense(128,activation='relu')(D1) #relu
D1 = Dense(32)(D1) 
Out = Dense(2, activation='softmax')(D1)

model = Model(inputs, Out)
model.summary()

opt = optimizers.Adam(learning_rate=0.0001, decay=1e-6) #0.002， 0.0005
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 8000, 24)]   0           []                               
                                                                                                  
 reshape_12 (Reshape)           (None, 200, 40, 24)  0           ['input_13[0][0]']               
                                                                                                  
 conv2d_96 (Conv2D)             (None, 200, 40, 64)  13888       ['reshape_12[0][0]']             
                                                                                                  
 conv2d_97 (Conv2D)             (None, 200, 40, 64)  36928       ['conv2d_96[0][0]']              
                                                                                           

#Single

In [ ]:
from torch.autograd.grad_mode import F
class EEGLoader(Dataset):
  def __init__(self,root):
    super(EEGLoader,self).__init__()
    self.root = root
    self.name2label = {}
    for name in sorted(os.listdir(os.path.join(root))):
      if not os.path.isdir(os.path.join(root,name)):
        continue
      self.name2label[name] = len(self.name2label.keys())
    print(self.name2label)

    # no enough data, so split two dataset manually. Just in case use shuffle,split will get only one or two samples for test 
    # assert len(data) == len(labels)


  # def load_csv(self,filename):
  #     data  = []
  #     for name in self.name2label.keys():
  #       print("name",name)
  #       data +=glob.glob(os.path.join(self.root,name,'*.csv'))
  #       # 。split(os.sep)[-2]
  #     print(len(data),data)


  def load_csv(self,foldername):
      # data  = []
      data =glob.glob(os.path.join(self.root,foldername,'*.csv'))
      # 。split(os.sep)[-2]
      print(len(data),data)

      res = []
      unusedChan = []
      for file in data:
        # remove dead people
        # if "Acquisition 09" in file : # or "Acquisition 30" in file
        #   continue

        EEGMatrix = pd.read_csv(file,header=None).values
        EEGwithoutConstantValue = []
        for index,channel in enumerate(EEGMatrix):
          # print("channel len",len(channel))
          # for testing since number of each acquisition's unique channels without 134733100 is different. get min number 29 rows first. Do data preprocessing to reduce channels to 17 later.
          if 134733100.0 in channel:
            if index not in unusedChan:
              unusedChan.append(index)
            continue
          EEGwithoutConstantValue.append(channel)
        # print(np.array(EEGwithoutConstantValue).shape)
        EEGwithoutConstantValue = EEGwithoutConstantValue[:24]
        # print(np.array(EEGwithoutConstantValue).shape)
        res.append(EEGwithoutConstantValue)
      print("unuse channel",unusedChan)
      return res
  

  def __len__(self):
      pass

  def __getitem__(self,idx):
      pass
data = EEGLoader('/content/drive/MyDrive/EEG')

resting_list2 = data.load_csv("01-single")
music_list2 = data.load_csv("08-single")

print(np.array(resting_list2).shape)
print(np.array(music_list2).shape)

{'01': 0, '01-health': 1, '01-previous': 2, '01-single': 3, '08': 4, '08-health': 5, '08-previous': 6, '08-single': 7, '16': 8, '16-health': 9, 'models': 10}
10 ['/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-3.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-5.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-6.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-8.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-9.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-7.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-2.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-4.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-1.csv', '/content/drive/MyDrive/EEG/01-single/Copy of Acquisition 09-rawTrigger1-10.csv']
unuse channel [1, 4, 5, 30,

In [ ]:
x_train_a = resting_list2[:6]
# .reshape((300,160,24))
x_train_a  = np.array(x_train_a)
x_train_a.shape

x_train_b = music_list2[:9]
# .reshape((300,160,24))
x_train_b = np.array(x_train_b)
x_train = np.vstack((x_train_a,x_train_b))
x_train.shape

x_train /= 200
x_train = x_train.astype("float32")

# test
x_test_a = resting_list2[6:]
# .reshape((150,160,24))
x_test_a  = np.array(x_test_a)
x_test_a.shape
print(len(x_test_a))

x_test_b = music_list2[9:]
# .reshape((150,160,24))
x_test_b = np.array(x_test_b)
x_test = np.vstack((x_test_a,x_test_b))
x_test.shape

x_test /= 200
x_test = x_test.astype("float32")
print(len(x_test))
# labels
y_train = np.append(np.zeros(len(x_train_a),dtype=int), np.ones(len(x_train_b),dtype=int))
y_train.shape

y_train = np_utils.to_categorical(y_train,2)
y_train = y_train.astype("float32")
y_train

y_test = np.append(np.zeros(len(x_test_a),dtype=int), np.ones(len(x_test_b),dtype=int))
y_test.shape


y_test = np_utils.to_categorical(y_test,2)
y_test = y_test.astype("float32")

4
9


In [ ]:
x_train = x_train.transpose(0,2,1)
x_test = x_test.transpose(0,2,1)
print(x_train.shape)
print(x_test.shape)

(15, 8000, 24)
(9, 8000, 24)


In [ ]:
from tensorflow.python.ops.gen_array_ops import Transpose
def residual_block(layer_input, filters):
  """Residual block described in paper"""
  d = Conv2D(filters, 3, activation='relu', padding='same')(layer_input)
  d = BatchNormalization(momentum=0.8)(d)
  d = Conv2D(filters, 3, activation='relu', padding='same')(d)
  d = BatchNormalization(momentum=0.8)(d)
  d = Add()([d, layer_input])
  return d




# cnn
# (192, 24, 8000) -> transpose 192 8000 24 -> reshape 192,20,400,24 
# 192,8000,2
# 变成(192,20,400,24)
inputs = Input(shape=(8000, 24))
c = Reshape((200,40,24))(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(c)
conv2 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
c2 = residual_block(conv2, 64)
#pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
c2 = residual_block(c2, 64)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(c2)
conv3 = Conv2D(32, 3, activation='relu', padding='same')(conv3)
D1 = Flatten()(conv3)
D1 = Dense(128,activation='relu')(D1) #relu
D1 = Dense(32)(D1) 
Out = Dense(2, activation='softmax')(D1)

model = Model(inputs, Out)
model.summary()

opt = optimizers.Adam(learning_rate=0.0001, decay=1e-6) #0.002， 0.0005
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 8000, 24)]   0           []                               
                                                                                                  
 reshape_1 (Reshape)            (None, 200, 40, 24)  0           ['input_2[0][0]']                
                                                                                                  
 conv2d_8 (Conv2D)              (None, 200, 40, 64)  13888       ['reshape_1[0][0]']              
                                                                                                  
 conv2d_9 (Conv2D)              (None, 200, 40, 64)  36928       ['conv2d_8[0][0]']               
                                                                                            

In [ ]:
history = model.fit(x_train,y_train,epochs=10,batch_size=3)

Epoch 1/10
5/5 [==============================] - 1s 28ms/step - loss: 128.3220 - accuracy: 0.3333
Epoch 2/10
5/5 [==============================] - 0s 27ms/step - loss: 14.0632 - accuracy: 0.6000
Epoch 3/10
5/5 [==============================] - 0s 20ms/step - loss: 3.0282 - accuracy: 0.7333
Epoch 4/10
5/5 [==============================] - 0s 20ms/step - loss: 8.7128 - accuracy: 0.8667
Epoch 5/10
5/5 [==============================] - 0s 19ms/step - loss: 7.9473e-08 - accuracy: 1.0000
Epoch 6/10
5/5 [==============================] - 0s 19ms/step - loss: 0.0879 - accuracy: 0.9333
Epoch 7/10
5/5 [==============================] - 0s 19ms/step - loss: 9.6711e-06 - accuracy: 1.0000
Epoch 8/10
5/5 [==============================] - 0s 20ms/step - loss: 0.4497 - accuracy: 0.9333
Epoch 9/10
5/5 [==============================] - 0s 19ms/step - loss: 1.1785 - accuracy: 0.8667
Epoch 10/10
5/5 [==============================] - 0s 19ms/step - loss: 3.3305 - accuracy: 0.9333


In [ ]:
from sklearn.metrics import f1_score
y_pred = model.predict(x_test)

1/1 [==============================] - 0s 130ms/step


In [ ]:
pred = np.int64(y_pred>0.1)
y_label = []
for v in y_test:
  y_label.append(v[0])
y_label=np.array(y_label,dtype=int)

pred_label = []
for v in pred:
  pred_label.append(v[0])
pred_label=np.array(pred_label,dtype=int)
pred_label

array([1, 1, 1, 1, 0, 0, 1, 1, 1])

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_label,pred_label)

0.7272727272727273

In [ ]:
y_pred

array([[1.0000000e+00, 1.0597195e-34],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 2.7222817e-34],
       [1.0000000e+00, 1.4494321e-19],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 8.5178055e-22],
       [1.0000000e+00, 1.4399747e-27],
       [1.0000000e+00, 3.7865323e-24]], dtype=float32)

In [ ]:
y_test

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)